# Proyecto Imágenes Laboratorio Inteligencia

### Profesor: Claudio Pérez
### Auxiliar: Juan Pérez
### Integrantes: Matías Osses - Ignacio - Alejandro

El objetivo de este proyecto final es desarrollar e implementar un algoritmo de búsqueda de imágenes similares basado en su contenido (Content Based Image Retrieval, CBIR) utilizando distintos métodos de extracción de características. Para el desarrollo de este proyecto debe descargar la base de datos INRIA Holidays dataset. Debe descargar los archivos jpg1.tar.gz y jpg2.tar.gz que contienen imágenes de consulta para 500 clases distintas y las imágenes que deberán ser almacenadas en el buscador, y que están asociadas a esas 500 clases. La figura 1 muestra las principales etapas de un sistema CBIR.


## Importación de librerías

In [1]:
import os
import copy
import time

import cv2
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import normalize

## Cálculo de vector de características

In [79]:
class Imagen():
    
    def __init__(self, path):
        self.nombre = path.split('\\')[-1]
        self.img = cv2.imread(path)
        self.img = cv2.cvtColor(self.img, cv2.COLOR_BGR2HSV)
        self.histogramas = dict()
        self.asd = dict()
        
    def division(self, n=3):
        nombre = '%sx%s'%(n,n)
        rows, cols, depth = self.img.shape
        histH = []
        histS = []
        histV = []
        div_y = np.linspace(0, rows, n+1).astype(int)
        div_x = np.linspace(0, cols, n+1).astype(int)
        for i, y in enumerate(div_y[1:]):
            for j, x in enumerate(div_x[1:]):
                celda = self.img[div_y[i]:div_y[i]+y, div_x[i]:div_x[i]+x, 0]
                histH.append(np.histogram(celda, bins=8, density = True)[0])
                celda = self.img[div_y[i]:div_y[i]+y, div_x[i]:div_x[i]+x, 1]
                histS.append(np.histogram(celda, bins=12, density = True)[0])
                celda = self.img[div_y[i]:div_y[i]+y, div_x[i]:div_x[i]+x, 2]
                histV.append(np.histogram(celda, bins=3, density = True)[0]) 
                
        self.histogramas[nombre+'_H'] = np.array(histH)
        self.histogramas[nombre+'_S'] = np.array(histS)
        self.histogramas[nombre+'_V'] = np.array(histV)
    
    def divisionNoRectangular(self):
        heigth, width, _ = self.img.shape
        center = (int(width/2), int(heigth/2))
        axis = (int(width/2*0.8), int(heigth/2*0.8))

        #elipse central
        container = np.zeros((heigth,width))
        r1 = (cv2.ellipse(container, center, axis, 0,0, 360, (255,255,255),-1)/255).astype(int)
        #esquina superior izquierda
        container = (container*0).astype(int)
        r2 = (r1 == 0).astype(int) * cv2.rectangle(container, (0,0), center, (255,255,255), -1)
        #esquina superior derecha
        container = (container*0).astype(int)
        r3 = (r1 == 0).astype(int) * cv2.rectangle(container, (int(width/2),0), (width, int(heigth/2) ), (255,255,255), -1)
        #esquina inferior izquierda
        container = (container*0).astype(int)
        r4 = (r1 == 0).astype(int) *  cv2.rectangle(container, (0,int(heigth/2)), (int(width/2), heigth ), (255,255,255), -1)
        #esquina inferior derecha
        container = (container*0).astype(int)
        r5 = (r1 == 0).astype(int) *  cv2.rectangle(container, center, (width,heigth), (255,255,255), -1)    
       
        
        canal = ['H', 'S', 'V']
        bins = [8, 12, 3]
        
        for (n,c,b) in zip(range(3),canal,bins):
            v1 = self.img[:,:,n][r1 > 0]
            v2 = self.img[:,:,n][r2 > 0]
            v3 = self.img[:,:,n][r3 > 0]
            v4 = self.img[:,:,n][r4 > 0]
            v5 = self.img[:,:,n][r5 > 0]
            self.histogramas['reg1_'+c] = np.histogram(v1, bins=b, density = True)[0]
            self.histogramas['reg2_'+c] = np.histogram(v2, bins=b, density = True)[0]
            self.histogramas['reg3_'+c] = np.histogram(v3, bins=b, density = True)[0]
            self.histogramas['reg4_'+c] = np.histogram(v4, bins=b, density = True)[0]
            self.histogramas['reg5_'+c] = np.histogram(v5, bins=b, density = True)[0]
    

    
    def calcularVectorCaracteristicas(self):
        self.division(3)
        self.division(4)
        self.division(6)
        self.divisionNoRectangular()
        
        h = np.array([],dtype=int)
        for key in self.histogramas.keys():
            h = np.concatenate([h,self.histogramas[key].reshape(-1)])
        return h
        

In [82]:
path = os.getcwd() + '\jpg'
img = path + '\\100401.jpg'

t = time.time()
a = Imagen(img)
hist = a.calcularVectorCaracteristicas()
t2 = time.time()
print(t2-t)


0.3139491081237793


In [83]:
hist.shape

(1518,)

## Distancias

In [4]:
def euclidiana(x,y):
    return np.sqrt(np.sum(x**2+y**2))

def coseno(x,y): 
    num = np.dot(x,y)
    den = np.sqrt((x**2).sum())*np.sqrt((y**2).sum())
    return num/den

def chi_cuadrado(x,y):
    x_t = x.sum()
    y_t = y.sum()
    c = x + y
    x_esperado = x_t*c/(x_t+y_t)
    y_esperado = y_t*c/(x_t+y_t)
    
    return (((x-x_esperado)**2)/x_esperado).sum() + (((y-y_esperado)**2)/y_esperado).sum()
    
def cramer(x,y):
    return(np.sqrt(chi_cuadrado(x,y)/(x.sum() + y.sum())))

## Content Based Image Retrieval (CBIR)

In [5]:
class CBIR():
    
    def __init__(self, distance):
        self.distance = distance
        self.database = list()
    
    def train(X):
        for image in X:
            self.database.append(Imagen(X).calcularVectorCaracteristicas())
    def evaluar(Input):
        scores = [self.distance(Input, x) for x in self.database]